In [1]:
# General import
import time
import math
import numpy as np
import pandas as pd
#from scipy import linalg
from collections import Counter

# local functions import
from functions import  *

In [2]:
split_data_tp1_app = open_file('../data/data_tp1_app.txt')
split_data_tp1_dec = open_file('../data/data_tp1_dec.txt')

split_data_tp2_app = open_file('../data/data_tp2_app.txt')
split_data_tp2_dec = open_file('../data/data_tp2_dec.txt')

split_data_tp3_app = open_file('../data/data_tp3_app.txt')
split_data_tp3_dec = open_file('../data/data_tp3_dec.txt')

In [3]:
def get_kppv_list(app_data, dec_point, k):
    dist_list = []
    for app_line in app_data:
        dist_list.append([app_line[0], compute_one_euclidian_dist(app_line[1:3], dec_point)])
    app_df = pd.DataFrame(dist_list)
    sorted_df = app_df.sort_values(by=1)
    kppv_list = sorted_df.head(k)
    return kppv_list

def compute_1ppv(app_data, dec_data):
    count_top_1 = 0
    classes_num =get_unique_class_num(app_data)
    conf_matrix = np.zeros((len(classes_num), len(classes_num)))
    for line in dec_data:
        kppv_list = get_kppv_list(app_data, line[1:], 1)
        if kppv_list.iloc[0][0] == line[0]:
            count_top_1 = count_top_1 + 1
        # Conf matrix
        row_num = int(line[0]) - 1
        col_num = int(kppv_list.iloc[0][0]) -1
        conf_matrix[row_num, col_num] = conf_matrix[row_num, col_num] + 1
    conf_matrix = transform_matrix_to_df(conf_matrix, classes_num)
    print_decision_model_result(len(app_data), len(dec_data), count_top_1/len(dec_data), top2_rate=None, conf_matrix=conf_matrix)

def get_majority_result(kppv_list):
    count_dict = Counter(kppv_list[0])
    temp = max(count_dict.values()) 
    res = [key for key in count_dict if count_dict[key] == temp] 
    return res

def get_k_cross_validation(app_data, k_max, cv):
    best_k = [0, 0]
    df = pd.DataFrame(app_data)
    shuffled = df.sample(frac=1)
    cut_dfs = np.array_split(shuffled, cv)  
    for i in range(k_max):
        k = i + 1
        sum_error = 0
        for i in range(cv):
            df_cv = cut_dfs[i]
            df_train = df.drop(df_cv.index)
            df_cv = df_cv.values.tolist()
            df_train = df_train.values.tolist()
            count_top_1 = 0
            for line in df_cv:
                kppv_list = get_kppv_list(df_train, line[1:], k)
                res = get_majority_result(kppv_list)
                if res[0] == line[0]:
                    count_top_1 = count_top_1 + 1
            error_rate = count_top_1/len(df_cv)
            sum_error = sum_error + error_rate
        if best_k[1] < sum_error/cv:
            best_k[0] = k
            best_k[1] = sum_error/cv
    return best_k

def compute_kppv(app_data, dec_data, k):
    count_top_1 = 0
    count_top_2 = 0
    classes_num =get_unique_class_num(app_data)
    conf_matrix = np.zeros((len(classes_num), len(classes_num)))
    for line in dec_data:
        kppv_list = get_kppv_list(app_data, line[1:], k)
        k_list = list(Counter(kppv_list[0]).items())
        k_list.sort(key=lambda a: a[1], reverse=True)
        if k_list[0][0] == line[0]:
            count_top_1 = count_top_1 + 1
            count_top_2 = count_top_2 + 1
        else:
            if len(k_list) >= 2:
                if k_list[1][0] == line[0]:
                    count_top_2 = count_top_2 + 1
        
    #     # Conf matrix
        row_num = int(line[0]) - 1
        col_num = int(k_list[0][0]) -1
        conf_matrix[row_num, col_num] = conf_matrix[row_num, col_num] + 1
    conf_matrix = transform_matrix_to_df(conf_matrix, classes_num)
    print_decision_model_result(len(app_data), len(dec_data), count_top_1/len(dec_data), count_top_2/len(dec_data), conf_matrix=conf_matrix)


In [4]:
print("First Data TP1")
start_time = time.time()
best_k = get_k_cross_validation(split_data_tp1_app, 15, 5)
compute_kppv(split_data_tp1_app, split_data_tp1_dec, best_k[0])
print("\tThe best k used is {} with a error rate = {}".format(best_k[0], best_k[1]))
print("\tExecution time : %s seconds\n" % (time.time() - start_time))
print("==========================\n")

print("First Data TP2")
start_time = time.time()
best_k = get_k_cross_validation(split_data_tp2_app, 15, 5)
compute_kppv(split_data_tp2_app, split_data_tp2_dec, best_k[0])
print("\tThe best k used is {} with a error rate = {}".format(best_k[0], best_k[1]))
print("\tExecution time : %s seconds\n" % (time.time() - start_time))
print("==========================\n")

print("First Data TP3")
start_time = time.time()
best_k = get_k_cross_validation(split_data_tp3_app, 15, 5)
compute_kppv(split_data_tp3_app, split_data_tp3_dec, best_k[0])
print("\tThe best k used is {} with a error rate = {}".format(best_k[0], best_k[1]))
print("\tExecution time : %s seconds\n" % (time.time() - start_time))
print("==========================\n")

First Data TP1
	Results :
	----------------
	Number of elements for the learning step :  500
	Number of elements for the decision step :  500
	----------------

	Top results :
	----------------
	Top 1 rate :  0.644
	Top 2 rate :  0.934
	----------------

	Confusion matrix :
	----------------
╒════╤═════╤═════╤═════╤═════╤═════╕
│    │   1 │   2 │   3 │   4 │   5 │
╞════╪═════╪═════╪═════╪═════╪═════╡
│  1 │  48 │  51 │   0 │   0 │   1 │
├────┼─────┼─────┼─────┼─────┼─────┤
│  2 │  45 │  52 │   0 │   0 │   3 │
├────┼─────┼─────┼─────┼─────┼─────┤
│  3 │   0 │   0 │  73 │  27 │   0 │
├────┼─────┼─────┼─────┼─────┼─────┤
│  4 │   0 │   0 │  41 │  55 │   4 │
├────┼─────┼─────┼─────┼─────┼─────┤
│  5 │   1 │   1 │   0 │   4 │  94 │
╘════╧═════╧═════╧═════╧═════╧═════╛
	----------------
	The best k used is 5 with a error rate = 0.626
	Execution time : 8.369944095611572 seconds


First Data TP2
	Results :
	----------------
	Number of elements for the learning step :  500
	Number of elements f